<a href="https://colab.research.google.com/github/ThalesCavalcanti/MachineLearningGroundwork/blob/main/Parte_5_5_To%CC%81picos_complementares_Selec%CC%A7a%CC%83o_de_atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

Reconecte ao Drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
DRIVE_DIRECTORY = "curso_ml"
DRIVE_DIRECTORY = os.path.join("/content/drive/MyDrive/cursoML", DRIVE_DIRECTORY)

---

In [4]:
import numpy as np
import pandas as pd
import pickle

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

## Seleção de atributos

### Base de dados cobertura vegetal

Neste exercício você vai mais uma vez utilizar a base de dados cobertura vegetal.

Carregue o arquivo csv salvo na pasta do Drive. Visualize o DataFrame resultante.

In [6]:
base = pd.read_csv(os.path.join(DRIVE_DIRECTORY, "cov_types.csv"))
base

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2767.0,66.0,17.0,210.0,18.0,1190.0,234.0,204.0,96.0,2251.0,2,30,Lodgepole Pine
1,2724.0,160.0,19.0,60.0,4.0,1350.0,236.0,240.0,127.0,2514.0,2,16,Lodgepole Pine
2,2360.0,65.0,7.0,127.0,21.0,1377.0,227.0,226.0,134.0,339.0,3,5,Ponderosa Pine
3,2995.0,45.0,4.0,285.0,30.0,5125.0,221.0,231.0,146.0,5706.0,0,11,Lodgepole Pine
4,2400.0,106.0,27.0,150.0,63.0,342.0,253.0,196.0,51.0,811.0,2,3,Ponderosa Pine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2917.0,90.0,9.0,247.0,25.0,4095.0,235.0,225.0,121.0,3901.0,0,28,Lodgepole Pine
9996,3015.0,38.0,8.0,361.0,74.0,4846.0,220.0,223.0,138.0,1611.0,0,28,Lodgepole Pine
9997,3052.0,79.0,19.0,90.0,11.0,1003.0,241.0,203.0,85.0,1490.0,2,22,Spruce/Fir
9998,2958.0,58.0,6.0,319.0,19.0,2468.0,225.0,227.0,137.0,2280.0,0,28,Lodgepole Pine


Desta vez você vai remover as colunas categóricas, pois a variância é mais informativa para atributos numéricos.

Primeiro, separe o nome das colunas em uma variável `colunas`, mas lembre-se de remover as 3 últimas colunas (que correspondem às 2 variáveis categóricas, mais a variável alvo).

In [7]:
columns = base.columns[:-3]
columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')

Agora crie as variáveis `X` e `y`, no formato de NumPy array, mas não inclua as variáveis categóricas em `X`.

In [8]:
X = base.iloc[:, :-3].values
y = base.iloc[:, -1].values


Exiba os valores de X.

In [9]:
X

array([[2767.,   66.,   17., ...,  204.,   96., 2251.],
       [2724.,  160.,   19., ...,  240.,  127., 2514.],
       [2360.,   65.,    7., ...,  226.,  134.,  339.],
       ...,
       [3052.,   79.,   19., ...,  203.,   85., 1490.],
       [2958.,   58.,    6., ...,  227.,  137., 2280.],
       [2682.,   91.,   13., ...,  219.,  108., 1661.]])

Crie um scaler do tipo `MinMaxScaler`, e aplique aos dados de `X`. Exiba o resultado.

In [10]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X

array([[0.4477459 , 0.18333333, 0.30909091, ..., 0.66666667, 0.384     ,
        0.31482517],
       [0.42571721, 0.44444444, 0.34545455, ..., 0.90666667, 0.508     ,
        0.35160839],
       [0.2392418 , 0.18055556, 0.12727273, ..., 0.81333333, 0.536     ,
        0.04741259],
       ...,
       [0.59375   , 0.21944444, 0.34545455, ..., 0.66      , 0.34      ,
        0.20839161],
       [0.54559426, 0.16111111, 0.10909091, ..., 0.82      , 0.548     ,
        0.31888112],
       [0.40420082, 0.25277778, 0.23636364, ..., 0.76666667, 0.432     ,
        0.23230769]])

Codifique os dados de `y` utilizando `LabelEncoder`.

In [11]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([4, 4, 5, ..., 6, 4, 4])

Por fim, salve as variáveis `X` e `y` na pasta do Drive, você vai utilizá-los nos exercícios seguintes.

In [12]:
with open(os.path.join(DRIVE_DIRECTORY, "cover_type_num.pkl"), "wb") as f:
    pickle.dump((X, y), f)

### Variância

Exiba a variância dos dados preditores. DICA: ao invés de um implementar um loop, você pode usar o método `var` com o parâmetro `axis=0`, que vai calcular a variância nas colunas.

In [13]:
X.var(axis=0)

array([0.02040914, 0.09559695, 0.01843345, 0.02361915, 0.0068116 ,
       0.05098971, 0.01885846, 0.01751286, 0.02366312, 0.03431677])

Perceba que temos 10 valores, correspondentes às 10 colunas numéricas do dataset.

Crie um seletor do tipo `VarianceThreshold` utilizando `threshold=0.02`. Exiba o `shape` do resultado.

In [14]:
selector = VarianceThreshold(threshold=0.02)
X_variance = selector.fit_transform(X)
X_variance.shape

(10000, 6)

Exiba as variâncias armazenadas no seletor.

In [15]:
selector.variances_

array([0.02040914, 0.09559695, 0.01843345, 0.02361915, 0.0068116 ,
       0.05098971, 0.01885846, 0.01751286, 0.02366312, 0.03431677])

Perceba que os resultados são os mesmos que você calculou antes.

Recupere os índices das colunas que foram selecionadas.

In [16]:
indices = np.where(selector.variances_ > 0.02)
indices

(array([0, 1, 3, 5, 8, 9]),)

Qual o nome das colunas selecionadas?

In [17]:
columns[indices]

Index(['Elevation', 'Aspect', 'Horizontal_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')

Agora, utilize os dados em `X_variance` e `y` para separar os dados em split de treinamento, e de teste (com `test_size=0.25`).

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_variance, y, test_size=0.25, random_state=0)

Crie uma instância do tipo `RandomForestClassifier`, com os argumentos padrão e `random_state=0`. Treine o algoritmo com os dados de treinamento.

In [19]:
tree_classifier = RandomForestClassifier(random_state=0)
tree_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=0)

Faça predições para os dados de teste, e calcule a acurácia.

In [20]:
y_pred = tree_classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.7712

Veja que neste caso, o algoritmo ficou um pouco pior que aquele treinado na **Parte 1**. Isso indica que, por mais que tenhamos acentuado os atributos com maior variância, e potencialmente por isso mais poder discriminativo, tanto os atributos de menor variância quanto os atributos categóricos têm influência não desprezível na classificação das árvores.

### Extra tree

Agora crie e treine uma instância do seletor `ExtraTreesClassifier`. Utilize todos os dados, ou seja, as variáveis `X`e `y`, na fase de seleção de atributos.

In [21]:
selector = ExtraTreesClassifier(random_state=0)
selector.fit(X, y)

ExtraTreesClassifier(random_state=0)

Exiba as importâncias determinadas pelo algoritmo.

In [22]:
selector.feature_importances_

array([0.3079157 , 0.06901482, 0.06347145, 0.0797224 , 0.07418725,
       0.10571598, 0.06438286, 0.06850195, 0.06323182, 0.10385578])

Com auxílio do NumPy, recupere os índices com importância maior que $0.07$.

In [23]:
indices = np.where(selector.feature_importances_ > 0.07)
indices

(array([0, 3, 4, 5, 9]),)

Que atributos foram selecionados?

In [24]:
columns[indices]

Index(['Elevation', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Horizontal_Distance_To_Fire_Points'],
      dtype='object')

Filtre os dados em `X` para recuperar somente os índices selecionados. Agora, observe que `indices` é uma tupla contendo um elemento, que é o array com os índices, então nesta construção você deve utilizar `indices[0]` para recuperar os índices dentro da tupla.

In [25]:
X_tree = X[:, indices[0]]

Mais uma vez, separe os dados `X_tree` e `y` em split de treinamento, e de teste (com `test_size=0.25`).

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_tree, y, test_size=0.25, random_state=0)

Crie uma instância do tipo `RandomForestClassifier`, com os argumentos padrão e `random_state=0`. Treine o algoritmo com os dados de treinamento.

In [27]:
tree_classifier = RandomForestClassifier(random_state=0)
tree_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

Faça predições para os dados de teste, e calcule a acurácia.

In [28]:
y_pred = tree_classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.7704

Mais uma vez esta seleção se mostrou pior que utilizar todos os dados, mas ficou um pouco melhor que a seleção baseada em variância.